In [3]:
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
import pathlib
data_root_orig = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_root = pathlib.Path(data_root_orig)
print(data_root)

/home/vitis-ai-user/.keras/datasets/flower_photos


In [4]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [5]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

3670

In [6]:
label_to_index = dict((name, index) for index, name in enumerate(label_names))
label_to_index
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]



In [7]:
print("First 10 labels indices: ", all_image_labels[:10])

First 10 labels indices:  [1, 2, 2, 3, 3, 2, 0, 1, 1, 4]


In [8]:
img_path = all_image_paths[0]
img_raw = tf.io.read_file(img_path)
print(repr(img_raw)[:100]+"...")
img_tensor = tf.image.decode_image(img_raw)

print(img_tensor.shape)
print(img_tensor.dtype)

img_final = tf.image.resize(img_tensor, [192, 192])
img_final = img_final/255.0

<tf.Tensor: shape=(), dtype=string, numpy=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00...
(240, 240, 3)
<dtype: 'uint8'>


In [9]:
def preprocess_image(image):
    
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [192, 192])
    image /= 255.0  # normalize to [0,1] range

    return image

def load_and_preprocess_image(path):
    image= tf.io.read_file(path)
    return preprocess_image(image)



In [10]:
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))
def load_and_preprocess_from_path_label(path, label):
      return load_and_preprocess_image(path), label
image_label_ds = ds.map(load_and_preprocess_from_path_label)
BATCH_SIZE = 32

# 设置一个和数据集大小一致的 shuffle buffer size（随机缓冲区大小）以保证数据
# 被充分打乱。
ds = image_label_ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# 当模型在训练的时候，`prefetch` 使数据集在后台取得 batch。
ds = ds.prefetch(buffer_size=AUTOTUNE)

ds = image_label_ds.apply(
  tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds = ds.batch(BATCH_SIZE)
ds = ds.prefetch(buffer_size=AUTOTUNE)

Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


In [11]:
import keras,os
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras_flops import get_flops

trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="cats_and_dogs_filtered/train",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="cats_and_dogs_filtered/validation", target_size=(224,224))
inputs = Input(shape=(192, 192, 3))

# 卷积层和最大池化层
conv1 = Conv2D(64, (3,3), padding='same', activation='relu')(inputs)
conv2 = Conv2D(64, (3,3), padding='same', activation='relu')(conv1)
pool1 = MaxPooling2D(pool_size=2)(conv2)

conv3 = Conv2D(128, (3,3), padding='same', activation='relu')(pool1)
conv4 = Conv2D(128, (3,3), padding='same', activation='relu')(conv3)
pool2 = MaxPooling2D(pool_size=2)(conv4)

conv5 = Conv2D(256, (3,3), padding='same', activation='relu')(pool2)
conv6 = Conv2D(256, (3,3), padding='same', activation='relu')(conv5)
conv7 = Conv2D(256, (3,3), padding='same', activation='relu')(conv6)
pool3 = MaxPooling2D(pool_size=2)(conv7)

conv8 = Conv2D(512, (3,3), padding='same', activation='relu')(pool3)
conv9 = Conv2D(512, (3,3), padding='same', activation='relu')(conv8)
conv10 = Conv2D(512, (3,3), padding='same', activation='relu')(conv9)
pool4 = MaxPooling2D(pool_size=2)(conv10)

conv11 = Conv2D(512, (3,3), padding='same', activation='relu')(pool4)
conv12 = Conv2D(512, (3,3), padding='same', activation='relu')(conv11)
conv13 = Conv2D(512, (3,3), padding='same', activation='relu')(conv12)
pool5 = MaxPooling2D(pool_size=2)(conv13)

# 扁平层
flat = Flatten()(pool5)

# 全联接层
fc1 = Dense(4096, activation='relu')(flat)
fc2 = Dense(4096, activation='relu')(fc1)

# 输出层
outputs = Dense(5, activation='softmax')(fc2)


my_VGG16_model = Model(inputs=inputs, outputs=outputs)
my_VGG16_model.summary()
flops = get_flops(my_VGG16_model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")

my_VGG16_model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy", 
              metrics=['accuracy'])

history = my_VGG16_model.fit(ds, epochs=5,steps_per_epoch=3)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 192, 192, 3)]     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 192, 192, 64)      1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 192, 192, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 96, 96, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 96, 96, 128)       73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 96, 96, 128)       147584    
_________________________________________

2022-03-03 13:31:45.259129: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-03-03 13:31:45.259261: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-03-03 13:31:45.263163: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.011ms.
  function_optimizer: function_optimizer did nothing. time = 0.002ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2022-03-03 13:32:02.758134: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


3/3 [==============================] - 58s 18s/step - loss: 7.7871 - accuracy: 0.1458
Epoch 2/5
3/3 [==============================] - 51s 17s/step - loss: 1.6020 - accuracy: 0.2812
Epoch 3/5
3/3 [==============================] - 53s 17s/step - loss: 1.8658 - accuracy: 0.2917
Epoch 4/5
3/3 [==============================] - 52s 17s/step - loss: 1.5902 - accuracy: 0.3438
Epoch 5/5
3/3 [==============================] - 53s 18s/step - loss: 1.6036 - accuracy: 0.2500
